In [1]:
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_boston
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [20]:
churndata = pd.read_csv('Customer-Churn.csv')
churndata.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


** Apply SMOTE for upsampling the data
    * Use logistic regression to fit the model and compute the accuracy of the model.
    * Use decision tree classifier to fit the model and compute the accuracy of the model.
    * Compare the accuracies of the two models.

In [21]:
#checking data types
churndata.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [22]:
#Spliting y and X

y = churndata['Churn']

numericals = churndata.select_dtypes(np.number)

In [23]:
y

0        No
1        No
2       Yes
3        No
4       Yes
       ... 
7038     No
7039     No
7040     No
7041    Yes
7042     No
Name: Churn, Length: 7043, dtype: object

In [24]:
numericals

,SeniorCitizen,tenure,MonthlyCharges
0,0,1,29.85
1,0,34,56.95
2,0,2,53.85
3,0,45,42.30
4,0,2,70.70
...,...,...,...
7038,0,24,84.80
7039,0,72,103.20
7040,0,11,29.60
7041,1,4,74.40


In [25]:
transformer = StandardScaler().fit(numericals)
X_stn = transformer.transform(numericals)
X_stn.shape

(7043, 3)

In [26]:
y.value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [28]:
smote = SMOTE()
X_sm, y_sm = smote.fit_resample(X_stn, y)
y_sm.value_counts()

No     5174
Yes    5174
Name: Churn, dtype: int64

In [ ]:
#LOGISTIC REGRESSION

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train,y_train)
preds = model.predict(X_test)

In [30]:
from sklearn.metrics import classification_report

In [31]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

          No       0.73      0.74      0.74      1021
         Yes       0.75      0.74      0.74      1049

    accuracy                           0.74      2070
   macro avg       0.74      0.74      0.74      2070
weighted avg       0.74      0.74      0.74      2070



In [ ]:
#DECISION TREE CLASSIFIER

In [32]:
from sklearn.tree import DecisionTreeClassifier

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=100)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
print(classification_report(y_test, preds))
score_sm_dec = round(model.score(X_test, y_test),2)

              precision    recall  f1-score   support

          No       0.76      0.78      0.77      1052
         Yes       0.76      0.74      0.75      1018

    accuracy                           0.76      2070
   macro avg       0.76      0.76      0.76      2070
weighted avg       0.76      0.76      0.76      2070



In [ ]:
#Comment: precision, recall and f1-score are higher on decission tree classifier

*Apply TomekLinks for downsampling
    *It is important to remember that it does not make the two classes equal but only removes the points from the majority class that are close to other points in minority class.

In [35]:
tomek = TomekLinks()
X_tl, y_tl = tomek.fit_resample(X_stn, y)
y_tl.value_counts()

No     4694
Yes    1869
Name: Churn, dtype: int64

In [ ]:
#LOGISTIC REGRESSION

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_tl, y_tl, test_size=0.2, random_state=100)
model = LogisticRegression()
model.fit(X_train,y_train)
preds = model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

          No       0.83      0.89      0.86       939
         Yes       0.67      0.55      0.60       374

    accuracy                           0.80      1313
   macro avg       0.75      0.72      0.73      1313
weighted avg       0.79      0.80      0.79      1313



In [ ]:
#DECISION TREE

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=100)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
print(classification_report(y_test, preds))
score_sm_dec = round(model.score(X_test, y_test),2)

              precision    recall  f1-score   support

          No       0.75      0.77      0.76      1052
         Yes       0.76      0.74      0.75      1018

    accuracy                           0.75      2070
   macro avg       0.75      0.75      0.75      2070
weighted avg       0.75      0.75      0.75      2070



In [40]:
tomek = TomekLinks()
X_tl, y_tl = tomek.fit_resample(X_stn, y)
y_tl.value_counts()

No     4694
Yes    1869
Name: Churn, dtype: int64